In [89]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 14: Extended Polymerization ---</h2><p>The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has <a href="https://en.wikipedia.org/wiki/Polymerization" target="_blank">polymerization</a> equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.</p>
<p>The submarine manual contains <span title="HOHO -> OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a <em>polymer template</em> and a list of <em>pair insertion</em> rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.</p>
<p>For example:</p>
<pre><code>NNCB

CH -&gt; B
HH -&gt; N
CB -&gt; H
NH -&gt; C
HB -&gt; C
HC -&gt; B
HN -&gt; C
NN -&gt; C
BH -&gt; H
NC -&gt; B
NB -&gt; B
BN -&gt; B
BB -&gt; N
BC -&gt; B
CC -&gt; N
CN -&gt; C
</code></pre>

<p>The first line is the <em>polymer template</em> - this is the starting point of the process.</p>
<p>The following section defines the <em>pair insertion</em> rules. A rule like <code>AB -&gt; C</code> means that when elements <code>A</code> and <code>B</code> are immediately adjacent, element <code>C</code> should be inserted between them. These insertions all happen simultaneously.</p>
<p>So, starting with the polymer template <code>NNCB</code>, the first step simultaneously considers all three pairs:</p>
<ul>
<li>The first pair (<code>NN</code>) matches the rule <code>NN -&gt; C</code>, so element <code><em>C</em></code> is inserted between the first <code>N</code> and the second <code>N</code>.</li>
<li>The second pair (<code>NC</code>) matches the rule <code>NC -&gt; B</code>, so element <code><em>B</em></code> is inserted between the <code>N</code> and the <code>C</code>.</li>
<li>The third pair (<code>CB</code>) matches the rule <code>CB -&gt; H</code>, so element <code><em>H</em></code> is inserted between the <code>C</code> and the <code>B</code>.</li>
</ul>
<p>Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.</p>
<p>After the first step of this process, the polymer becomes <code>N<em>C</em>N<em>B</em>C<em>H</em>B</code>.</p>
<p>Here are the results of a few steps using the above rules:</p>
<pre><code>Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
</code></pre>
<p>This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, <code>B</code> occurs 1749 times, <code>C</code> occurs 298 times, <code>H</code> occurs 161 times, and <code>N</code> occurs 865 times; taking the quantity of the most common element (<code>B</code>, 1749) and subtracting the quantity of the least common element (<code>H</code>, 161) produces <code>1749 - 161 = <em>1588</em></code>.</p>
<p>Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>
</article>


In [90]:
from collections import Counter

from more_itertools import minmax


example = """
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
"""


def polymerization(s: str, steps=10, do_print=False):
    template, pair_insertions = re.split(r"\n\s*\n", s.strip())
    template = list(template)
    pair_insertions = dict(pi.split(" -> ") for pi in pair_insertions.splitlines())

    if do_print:
        print(f"template\t\t: {''.join(template)}")

    for step in range(1, steps + 1):
        new_template = []
        for i in range(len(template) - 1):
            new_template.append(template[i])
            pair = "".join(template[i : i + 2])
            if pair in pair_insertions:
                new_template.append(pair_insertions[pair])
        new_template.append(template[-1])
        template = new_template

        if do_print:
            print(f"After step {step}\t\t: {''.join(template)}")

    mn, mx = minmax(Counter(template).values())
    return mx - mn


assert polymerization(example) == 1588

In [91]:
with open("../input/day14.txt") as f:
    puzzle = f.read()

print(f"Part I: { polymerization(puzzle) }")

Part I: 3306


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3306</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of <em>40 steps</em> should do it.</p>
<p>In the above example, the most common element is <code>B</code> (occurring <code>2192039569602</code> times) and the least common element is <code>H</code> (occurring <code>3849876073</code> times); subtracting these produces <code><em>2188189693529</em></code>.</p>
<p>Apply <em>40</em> steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>
</article>

</main>


In [92]:
def transform(s: str, pair_insertions: dict[str, str]) -> list[str]:
    result = []
    for i in range(len(s) - 1):
        result.append(s[i])
        pair = s[i : i + 2]
        if pair in pair_insertions:
            result.append(pair_insertions[pair])
    result.append(s[-1])
    return ["".join(result[i - 1 : i + 1]) for i in range(1, len(result))]


def polymerization_fast(s: str, steps=10):
    template, pair_insertions = re.split(r"\n\s*\n", s.strip())
    pair_insertions = dict(pi.split(" -> ") for pi in pair_insertions.splitlines())

    pairs = Counter(template[i - 1 : i + 1] for i in range(1, len(template)))

    for _ in range(steps):
        new_pairs = defaultdict(int)
    
        for p, count in pairs.items():
            for pp in transform(p, pair_insertions):
                new_pairs[pp] += count

        pairs = new_pairs

    letter_count = defaultdict(int)
    for k, v in pairs.items():
        for ch in k:
            letter_count[ch] += v

    mn, mx = minmax(letter_count.values())
    return (mx - mn) // 2 + 1


assert polymerization_fast(example, 40) == 2188189693529

In [93]:
print(f"Part II: { polymerization_fast(puzzle,40) }")

Part II: 3760312702877


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3760312702877</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
